In [1]:
data_var = '2023-12-26'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7021,2023-12-26,França Lnb,16:00,Boulogne-Levallois,Monaco,7.00,1.10,162.5,1.88,1.90,11.5,2.10,1.68,QJRon2Dq,0.142857,0.909091,0.531915,0.526316,0.051948,300.824,190.398356,0.632923,0.6,1.341641,2.236068,243.60,3.904,2.614332,0.669655,-55.0,115.312,17.882605,0.155080,3.0,0.000000,0.000000,128.76,1.242,0.155467,0.125175,101.0,84,116,2.90,1.11,151.266,0.000,1.030106,0.007483,0.157135,-2.97,-0.594,-10.101010,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7022,2023-12-26,França Lnb,13:30,Strasbourg,Chalon/Saone,1.39,3.03,157.5,1.81,1.95,-7.5,2.04,1.74,G8fvhh12,0.719424,0.330033,0.552486,0.512821,0.049457,168.902,108.427525,0.641955,1.8,1.643168,0.912871,358.45,2.426,1.668017,0.687558,6.0,203.736,122.189922,0.599746,1.2,1.643168,1.369306,128.77,2.670,1.809240,0.677618,0.0,67,79,5.35,1.63,111.844,350.462,0.524731,0.052657,0.112239,0.51,0.102,3.823529,0.0,0.0,0.0,-5.00,-1.000,-2.030000,0.0,0.0,0.0
7023,2023-12-26,Europa Liga Bnxt,16:30,Oostende,Antwerp Giants,1.48,2.53,150.5,1.85,1.85,-5.5,2.01,1.72,C8JKXe7L,0.675676,0.395257,0.540541,0.540541,0.070933,97.776,14.851893,0.151897,3.0,0.000000,0.000000,107.92,1.240,0.165378,0.133370,67.0,137.168,53.711722,0.391576,3.0,0.000000,0.000000,140.06,1.538,0.715171,0.465001,123.0,76,94,1.42,1.49,0.000,0.000,0.370305,0.000000,0.109952,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7024,2023-12-26,Europa Liga Bnxt,16:30,Okapi Aalst,Belfius Mons,1.48,2.53,151.5,1.83,1.87,-5.5,2.02,1.71,xjR7zbN2,0.675676,0.395257,0.546448,0.534759,0.070933,286.316,178.178410,0.622314,0.0,0.000000,NaN,321.12,3.986,2.328472,0.584162,-75.0,262.996,134.668925,0.512057,0.6,1.341641,2.236068,280.28,4.384,3.040087,0.693451,-64.0,72,91,4.46,3.08,0.000,0.000,0.370305,0.015289,0.117535,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7025,2023-12-26,Europa Liga Bnxt,16:30,Mechelen,Limburg,1.72,2.02,146.5,1.87,1.79,-2.5,1.98,1.74,b5FGYyhF,0.581395,0.495050,0.534759,0.558659,0.076445,110.160,42.884039,0.389289,1.2,1.643168,1.369306,107.20,1.494,0.332235,0.222379,-24.0,96.478,9.928183,0.102906,1.8,1.643168,0.912871,92.66,1.280,0.118322,0.092439,37.0,80,82,1.34,1.13,0.000,0.000,0.113440,0.030912,0.091240,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7079,2023-12-26,Áustria Superliga,15:15,Wels,Swans Gmunden,1.85,1.87,157.5,1.85,1.85,-1.5,1.99,1.73,4v603lXB,0.540541,0.534759,0.540541,0.540541,0.075300,111.646,17.797808,0.159413,1.8,1.643168,0.912871,124.20,1.440,0.293002,0.203473,53.0,116.902,53.149693,0.454652,1.2,1.643168,1.369306,184.80,1.486,0.524481,0.352948,-21.0,69,80,1.80,2.31,0.000,0.000,0.007603,0.000000,0.098843,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7080,2023-12-26,Bahrein Premier League,13:30,Bahrain Club,Samaheej,2.93,1.13,158.5,1.86,1.84,1.5,1.98,1.74,E7zYBTjs,0.341297,0.884956,0.537634,0.543478,0.226253,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,61,83,13.20,1.59,0.000,0.000,0.626991,0.007644,0.091240,0.00,0.000,inf,0.0,0

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V1 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7053,00:30,Eua Nba,Phoenix Suns,Dallas Mavericks,1.6,Back Home


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
7025,16:30,Europa Liga Bnxt,Mechelen,Limburg,1.72,Back Home
7036,13:40,Israel Liga Leumit,Maccabi Raanana,Elitzur Ashkelon,1.98,Back Home
7046,14:00,Eslovênia Liga Nova Kbm,LTH Castings,Helios Domzale,2.37,Back Home
7047,15:00,Eslovênia Liga Nova Kbm,Podcetrtek,Sencur,1.75,Back Home
7053,00:30,Eua Nba,Phoenix Suns,Dallas Mavericks,1.60,Back Home
7056,22:00,Eua Nba,Chicago Bulls,Atlanta Hawks,1.84,Back Home
7057,22:00,Eua Nba,Houston Rockets,Indiana Pacers,1.72,Back Home
7059,22:00,Eua Nba,Oklahoma City Thunder,Minnesota Timberwolves,1.61,Back Home
7060,22:00,Eua Nba,San Antonio Spurs,Utah Jazz,2.30,Back Home
7061,15:30,Europa Eurocopa,JL Bourg,Gran Canaria,1.80,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7058,22:00,Eua Nba,New Orleans Pelicans,Memphis Grizzlies,1.58,Back Home
7068,14:30,Alemanha Bbl,Syntainics MBC,Vechta,2.33,Back Home
7079,15:15,Áustria Superliga,Wels,Swans Gmunden,1.85,Back Home
